# ETL Read and Manupulation

In [ ]:
# ETL_READ_PONTOS_PRODUTOS

_df = etl_pre_read_pontos_produtos

#create column time
_mes_dict = {1: '01', 2: '02', 3: '03', 4: '04', 5: '05', 6: '06', 7:'07', 8: '08', 9: '09', 10: '10', 11: '11', 12: '12'}
_df['Mes'] = _df['Mes'].replace(_mes_dict)
_df['Ano'] = _df['Ano'].astype(str)
_df['time'] = _df['Ano'] + '.' + _df['Mes']
_df = _df.drop(columns = ['Ano', 'Mes'])

try:
    _df['id_produto'] = _df['id_produto'].astype(int)
except:
    pass
_df['id_produto'] = _df['id_produto'].astype(str)

_df['pais_emitente'] = _df['pais_emitente'].str.upper()

_df.columns = ['pais', 'cod_produto', 'pto_quali', 'pto_boni', 'time']


_df = _df[_df['time']<current_month]


result = _df

In [ ]:
# spark_session
from pyspark.sql import SparkSession
if SparkSession._activeSession:
    SparkSession._activeSession.stop()
result = SparkSession.builder.master('local[*]') \
    .config("spark.local.dir", "/data/tmp")\
    .appName("TestApp").getOrCreate()

In [ ]:
# ETL_READ_SELLOUT_BU

import pyspark.sql.functions as sf
from pyspark.sql.functions import col , udf

_df = etl_read_sellout
_df = _df.withColumnRenamed('cod_produto', 'id_produto')

# ### reasign kits to product
_df3 = read_kit_produto.copy()
_df3 = _df3[['pais', 'cod_kit','cod_produto', 'quantidade']]#, 'descricao' ]]
_df3 = _df3.rename(columns = {'cod_kit':'id_produto'})#, 'descricao': 'descricao_prod'})
_df3 = spark_session.createDataFrame(_df3) 
_df = _df.join(_df3, on = (_df.pais== _df3.pais)&(_df.id_produto == _df3.id_produto), how = 'left').drop(_df3.pais).drop(_df3.id_produto)

_df = _df.na.fill(0.0, 'receita_sellout_real')
_df = _df.na.fill(0.0, 'Volume_sellout')
_df = _df.na.fill(1, 'quantidade')

_df = _df.withColumn('volume', sf.col('Volume_sellout')*sf.col('quantidade'))

_df = _df.withColumn('cod_produto', sf.when(sf.col('cod_produto').isNull(), sf.col('id_produto')).otherwise(sf.col('cod_produto')))

# _df['descr_produto'] = np.where(_df['descricao_prod'].notnull(), _df['descricao_prod'], _df['descr_produto'])

#create column time
_mes_dict = {1: '01', 2: '02', 3: '03', 4: '04', 5: '05', 6: '06', 7:'07', 8: '08', 9: '09', 10: '10', 11: '11', 12: '12'}

map_func = udf(lambda row : _mes_dict.get(row,row))
_df = _df.withColumn("Mes", map_func(col("Mes")))

_df = _df.select(sf.concat_ws('.', _df.Ano, _df.Mes).alias('time'), sf.col("*"))

_df = _df.withColumn('volume',sf.col('volume').cast("float").alias('volume'))
_df = _df.withColumn('receita_sellout_real',sf.col('receita_sellout_real').cast("float").alias('receita_sellout_real'))

_df = _df.groupby('time', 'pais', 'cod_produto').agg(sf.sum("volume").alias("volume"),sf.sum("receita_sellout_real").alias("receita"))

_df = _df.filter(_df.time<current_month)
     
result=_df.toPandas()

In [1]:
# Datalake Service Client

from azure.storage.filedatalake import DataLakeServiceClient
from azure.identity import ClientSecretCredential     

_account_url = 'https://datalake.dfs.core.windows.net'
_client_id = '0c03'
_tenant_id = '0a74a2'
_client_secret = '9018Q'

_credentialA = ClientSecretCredential(_tenant_id, _client_id, _client_secret)

_service_client = DataLakeServiceClient(account_url=_account_url, credential=_credentialA)

result = _service_client

In [ ]:
# Datalake File System Client

result = etl_datalake_service_client.get_file_system_client(file_system='pyplan')

In [ ]:
# Read paths

import os 

fileSystemClient=etl_datalake_file_system_client
filesPath = os.path.join('consultores')
# filesPath = None

_f = fileSystemClient.get_paths(path=filesPath, recursive=True)

for f in _f:
    print(f)

result = ''

In [ ]:
# List Directory Example

path_list = etl_datalake_file_system_client.get_paths( path= "/", recursive=False, max_results=500)

res = []
for path in path_list: 
    res.append(path.name)
    print(path.name)
    print("read_" + path.name + '\n')

result = res[len(res)-1]